# Hosting
Once the training is done, we can deploy the trained model as an Amazon SageMaker hosted endpoint. This will allow us to make predictions (or inference) from the model. Note that we don't have to host on the same instance (or type of instance) that we used to train. Training is a prolonged and compute heavy job that require a different of compute and memory requirements that hosting typically do not. We can choose any sagemaker supported instance we want to host the model. The notebooks walks through the endpoint deployment.:

## Setup
The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting. If you don't specify a bucket, SageMaker SDK will create a default bucket following a pre-defined naming convention in the same region. The IAM role ARN used to give SageMaker access to your data. It can be fetched using the get_execution_role method from sagemaker python SDK if running this notebook in sagemaker studio

profile = aws profile
role = predefined role arn

In [10]:
import sagemaker
import boto3
import time
from PIL import Image

role='arn:aws:iam::395166463292:role/service-role/AmazonSageMaker-ExecutionRole-20200714T182988'

profile = 'sites'
region_name='us-east-2'
bucket = 'st-crayon-dev'
prefix = 'sagemaker/labelbox/'

session = boto3.session.Session(profile_name = profile, region_name = region_name)
sess = sagemaker.Session(session,default_bucket=bucket)
sagemaker_client = session.client('sagemaker')
print(sess.boto_session)

Session(region_name='us-east-2')


## Training image
Since we are using prebaked aws semantic segmentation algo, we need the Amazon SageMaker Semantic Segmentaion docker image, which is static and need not be changed

In [11]:
from sagemaker.amazon.amazon_estimator import get_image_uri
training_image = get_image_uri(sess.boto_region_name, 'semantic-segmentation', repo_version="latest")
print (training_image)

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
825641698319.dkr.ecr.us-east-2.amazonaws.com/semantic-segmentation:latest


In [12]:
!aws sagemaker list-training-jobs --name-contains ss-labelbox-train-aug-manifest --status-equals Completed --profile $profile

{
    "TrainingJobSummaries": [
        {
            "TrainingJobName": "ss-labelbox-train-aug-manifest-15965354-2020-08-04-10-18-20-513",
            "TrainingJobArn": "arn:aws:sagemaker:us-east-2:395166463292:training-job/ss-labelbox-train-aug-manifest-15965354-2020-08-04-10-18-20-513",
            "CreationTime": 1596536301.162,
            "TrainingEndTime": 1596541653.786,
            "LastModifiedTime": 1596541653.786,
            "TrainingJobStatus": "Completed"
        },
        {
            "TrainingJobName": "ss-labelbox-train-aug-manifest-15965315-2020-08-04-08-59-08-997",
            "TrainingJobArn": "arn:aws:sagemaker:us-east-2:395166463292:training-job/ss-labelbox-train-aug-manifest-15965315-2020-08-04-08-59-08-997",
            "CreationTime": 1596531549.702,
            "TrainingEndTime": 1596533214.363,
            "LastModifiedTime": 1596533214.363,
            "TrainingJobStatus": "Completed"
        },
        {
            "TrainingJobName": "ss-labelbox-train-

## Create a deployable model 
by identifying the location of model artifacts and the Docker image that contains the inference code.

In [15]:
model_name = f'ss-labelbox-1024-v0'
training_job_name = 'ss-labelbox-train-aug-manifest-15964472-2020-08-03-09-49-26-446'
info = sagemaker_client.describe_training_job(TrainingJobName=training_job_name)
model_data = info['ModelArtifacts']['S3ModelArtifacts']
print(model_data)
primary_container = {
    'Image': training_image,
    'ModelDataUrl': model_data
}
create_model_response = sagemaker_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])

s3://st-crayon-dev/sagemaker/labelbox/output/ss-labelbox-train-aug-manifest-15964472-2020-08-03-09-49-26-446/output/model.tar.gz
arn:aws:sagemaker:us-east-2:395166463292:model/ss-labelbox-1024-v0


## Create an Amazon SageMaker endpoint configuration 
by specifying the ML compute instances that you want to deploy your model to.

In [16]:
endpoint_config_name = f'ss-labelbox-1024-config-v0'
print(endpoint_config_name)
create_endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.c4.2xlarge',
        'InitialVariantWeight':1,
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

ss-labelbox-1024-config-v0
Endpoint Config Arn: arn:aws:sagemaker:us-east-2:395166463292:endpoint-config/ss-labelbox-1024-config-v0


## Create an Amazon SageMaker endpoint.

In [18]:
import time

endpoint_name = f'ss-labelbox-1024-size-1024-v0'
print(endpoint_name)
create_endpoint_response = sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

resp = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Status: " + status)

ss-labelbox-1024-size-1024-v0
arn:aws:sagemaker:us-east-2:395166463292:endpoint/ss-labelbox-1024-size-1024-v0
Status: Creating


## Wait for model deployment
once model is deployed Status changes to 'InService'

In [19]:
while status=='Creating':
    time.sleep(60)
    resp = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']
    print("Status: " + status)

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-2:395166463292:endpoint/ss-labelbox-1024-size-1024-v0
Status: InService
